In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
import _pickle as pickle
import numpy as np
import nltk 
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as st
from datetime import date

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

In [3]:
import time
import warnings
warnings.filterwarnings('ignore')

In [4]:
repo_dir = '/home/a12884/repos/Deep-Learning-Financial-News-Stock-Movement-Prediction/xingyou_song_event_embeddings'

In [5]:
with open(repo_dir + '/word_embeddings/run_info.p', 'rb') as f:
    x = pickle.load(f, encoding='bytes')

In [6]:
xx = dict()
for k,v in x.items():
    print(type(k), type(v))
    if isinstance(v, np.ndarray):
        xx[k.decode('utf8')] = v
    elif isinstance(v, list):
        xx[k.decode('utf8')] = map(lambda z: z.decode('utf8'), v)
    else:
        vv = dict()
        for k2,v2 in v.items():
            new_k2 = k2
            if isinstance(k2, tuple):
                new_k2 = tuple(map(lambda z: z.decode('utf8'), k2))
            elif isinstance(k2, bytes):
                new_k2 = k2.decode('utf8')
            vv[new_k2] = v2
        xx[k.decode('utf8')] = vv

<class 'bytes'> <class 'dict'>
<class 'bytes'> <class 'numpy.ndarray'>
<class 'bytes'> <class 'list'>
<class 'bytes'> <class 'list'>
<class 'bytes'> <class 'dict'>
<class 'bytes'> <class 'dict'>


In [7]:
xx.keys()

dict_keys(['info2index', 'We_mat', 'word_emb', 'index2info', 'word2idx', 'We_dict'])

In [8]:
info2index = xx['info2index']

In [9]:
counter = 0
for key_ in info2index:
    print(key_)
    print(info2index[key_])
    counter += 1
    if counter == 5:
        break

('AAPL', '20141009', 'icahn urges apple to repurchase more shares.')
0
('AAPL', '20170207', 'channeling steve jobs apple seeks design perfection at new paceship campus.')
1
('GOOGL', '20170607', 'lack of power lines hits subsaharan africa biggest wind farm.')
2
('IBM', '20170419', 'ibm cloud provides little silver lining.')
3
('AAPL', '20140507', 'ting hsin foxconn submit rival bids for taiwan asia pacific telecom source.')
4


In [10]:
event_embeddings = pd.read_csv(repo_dir + "/word_embeddings/u_epoch_500.csv", header = None)
embedding_lst = []
for row in event_embeddings.iterrows():
    index, data = row 
    temp = data.tolist()
    actual_data = [float(x) for x in temp[0].split()]
    embedding_lst.append(actual_data)

In [11]:
number_to_month = {"01": "Jan", "02":"Feb", "03":"Mar", "04":"Apr", "05":"May", "06": "Jun", "07":"Jul", "08":"Aug", "09":"Sep", "10":"Oct", "11":"Nov", "12":"Dec"}
def conv_num_to_string(d): #ex: conv_num_to_string('20041001') = '01-Oct-04'
    year = d[0:4]
    month = d[4:6]
    day = d[6:8]
    new = day + "-" + number_to_month[month] + "-" + year[2:4]
    return new 

def numeric_day_distance(day1, day2): #'20140111', '20150115'
    d0 = date(int(day1[0:4]), int(day1[4:6]), int(day1[6:8]) )
    d1 = date(int(day2[0:4]), int(day2[4:6]), int(day2[6:8]) )
    delta = d0 - d1 
    return abs(delta.days)

In [12]:
stock_to_events = {}
for key_ in info2index:
    stock_ = key_[0]
    embedding_to_index = info2index[key_]
    date_ = key_[1]
    event = key_[2]
    new_value = [date_, embedding_to_index]
    if stock_ in stock_to_events: 
        stock_to_events[stock_].append(new_value)
    else:
        stock_to_events[stock_] = [new_value]

for stock_ in stock_to_events:
    stock_to_events[stock_] = sorted( stock_to_events[stock_], key = lambda x: x[0]  )

In [13]:
stock_to_events.keys()

dict_keys(['AAPL', 'GOOGL', 'IBM', 'AMZN', 'CSCO', 'NVDA', 'MSFT', 'QCOM', 'INTC', 'AMD'])

In [14]:
stock_to_events['GOOGL'][:5]

[['20140403', 2209],
 ['20140403', 3830],
 ['20140403', 6016],
 ['20140404', 1684],
 ['20140407', 2966]]

In [15]:
stk_lst_0 = []

for stock in stock_to_events:
    stock_length = {}
    stock_length["Number of Articles"] = len(stock_to_events[stock])
    stock_length["Stock Name"] = stock
    stk_lst_0.append(stock_length)

In [16]:
df = pd.DataFrame(stk_lst_0)
df[ ["Stock Name", "Number of Articles"]]

,Stock Name,Number of Articles
0,AAPL,2292
1,GOOGL,528
2,IBM,415
3,AMZN,1062
4,CSCO,229
5,NVDA,54
6,MSFT,830
7,QCOM,351
8,INTC,409
9,AMD,23


In [17]:
news_csv = pd.read_csv(repo_dir + "/news_data/news_reuters_10.csv", error_bad_lines=False, header = None, names = ["stock", "company", "date", "title", "summary", "type", "website"])
google_price_csv = pd.read_csv(repo_dir + "/price_data/GOOGL_2006-01-01_to_2017-11-01.csv")

In [18]:
def up_down_ratio(stock, day_lag): #ex: sentiment_to_price_plot("AAPL", 1, 'neg')
    stock_data = news_csv[news_csv["stock"] == stock]
    stock_price_csv = pd.read_csv(repo_dir + "/price_data/"+ stock+"_2006-01-01_to_2017-11-01.csv")
    total = []
    for index, row in stock_data.iterrows():
    

        day = conv_num_to_string(str(row["date"]) )

        if day in stock_price_csv["Date"].values:

            

            row_index = stock_price_csv.index[stock_price_csv["Date"] == day].tolist()[0]
            next_price = stock_price_csv.iloc[row_index - day_lag  ]
            #print next_price["Date"], google_price_csv.iloc[row_index]["Date"]
            diff = next_price["Close"] - next_price["Open"]
            if diff >= 0.0:
                total.append(1) 
            else:
                total.append(0)
    return 100*sum(total)/len(total)

In [19]:
%%time
stk_lst = []

for stock in stock_to_events:
    if stock != 'IBM':
        stock_length = {}
        stock_length["Price Up Percentage"] = up_down_ratio(stock, 1)
        stock_length["Stock Name"] = stock
        stk_lst.append(stock_length)

CPU times: user 11.6 s, sys: 215 ms, total: 11.9 s
Wall time: 11.5 s


In [20]:
df = pd.DataFrame(stk_lst)
df[["Stock Name", "Price Up Percentage"]]

,Stock Name,Price Up Percentage
0,AAPL,48.922958
1,GOOGL,49.317406
2,AMZN,48.059965
3,CSCO,60.096931
4,NVDA,72.413793
5,MSFT,51.516566
6,QCOM,45.331070
7,INTC,49.537513
8,AMD,50.000000


In [21]:
def total_embedding_to_class(stock, training_ratio, shuff_bool): 


    start_time = time.time()

    #----------------------------LAGS
    day_lag = 1 
    week_lag = 7
    month_lag = 30
    #--------------------------- NN parameters
    input_size = 100
    window_size_convM =3
    hidden_size_convM = 20

    window_size_convL =3 
    hidden_size_convL = 40


    hidden_size_end = 200

    learning_rate = 0.001
    num_epochs = 500
    batch_size = 50

    #training_ratio = 0.8
    #stock = 'AAPL'

    #----------------------------------- PARAMTETRS


    stock_price_csv = pd.read_csv(repo_dir + "/price_data/"+ stock+"_2006-01-01_to_2017-11-01.csv")

    temp_x = []
    temp_y = []
    for i in range(len(stock_to_events[stock]) ):
        event = stock_to_events[stock][i]


        date_numeric = event[0]
        day = conv_num_to_string( date_numeric    )

        if day in stock_price_csv["Date"].values:

            temp = {}
            temp["day"] = embedding_lst[event[1]]



            row_index = stock_price_csv.index[stock_price_csv["Date"] == day].tolist()[0]
            next_price_data = stock_price_csv.iloc[row_index - day_lag  ]

            next_price = next_price_data["Close"] - next_price_data["Open"]

            if next_price >= 0.0:
                pos_neg_class = 1
            else:
                pos_neg_class = 0
            temp_y.append(pos_neg_class)



            temp["week"] = []
            temp_date_before = 1
            while ( i - temp_date_before >= 0 and numeric_day_distance(stock_to_events[stock][i-temp_date_before][0], date_numeric ) <= week_lag ) :
                temp['week'].append(embedding_lst[stock_to_events[stock][i-temp_date_before][1]]    )
                temp_date_before +=1 

            temp["month"] = []
            temp_date_before = 1
            while ( i - temp_date_before >= 0 and numeric_day_distance(stock_to_events[stock][i-temp_date_before][0], date_numeric ) <= month_lag ) :
                temp['month'].append(embedding_lst[stock_to_events[stock][i-temp_date_before][1]]    )
                temp_date_before +=1 
            temp_x.append(temp)
    #--------------------------------------

    print ("Generated event embedded data.")
    elapsed_time = time.time() - start_time
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")



    #--------------------------
    sample_size = len(temp_x)
    cut = int(training_ratio*float(sample_size) ) 
    train_x = temp_x[0:cut ]
    test_x = temp_x[cut+1:]

    train_y = temp_y[0:cut]
    test_y = temp_y[cut+1:]


    max_event_length_week = max([len(day_embedding["week"]) for day_embedding in temp_x ])
    max_event_length_month = max([len(day_embedding["month"]) for day_embedding in temp_x ])


    train_x_concatenate = []



    for day_embedding in train_x: 
        block = [day_embedding["day"]]
        week_padding_number = max_event_length_week - len(day_embedding["week"])
        block = block + day_embedding["week"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(week_padding_number)]

        month_padding_number = max_event_length_month - len(day_embedding["month"])
        block = block + day_embedding["month"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(month_padding_number)]


        train_x_concatenate.append(block)

    train_x_concatenate = torch.FloatTensor(train_x_concatenate)

    train_y = torch.LongTensor(train_y)
    train_x_concatenate_temp = torch.utils.data.TensorDataset(train_x_concatenate, train_y)
    train_loader_total = torch.utils.data.DataLoader(dataset=train_x_concatenate_temp, 
                                           batch_size = batch_size, 
                                           shuffle=shuff_bool)



    #-----------------------------------

    test_x_concatenate = []


    for day_embedding in test_x: 
        block = [day_embedding["day"]]
        week_padding_number = max_event_length_week - len(day_embedding["week"])
        block = block + day_embedding["week"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(week_padding_number)]

        month_padding_number = max_event_length_month - len(day_embedding["month"])
        block = block + day_embedding["month"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(month_padding_number)]


        test_x_concatenate.append(block)


    test_x_concatenate = torch.FloatTensor(test_x_concatenate)

    test_y = torch.LongTensor(test_y)
    test_x_concatenate_temp = torch.utils.data.TensorDataset(test_x_concatenate, test_y)
    test_loader_total = torch.utils.data.DataLoader(dataset=test_x_concatenate_temp, 
                                           batch_size = batch_size, 
                                           shuffle=False)



    #------------------------------------------
    class Net(nn.Module):

        def __init__(self, input_size, window_size_convM, hidden_size_convM, window_size_convL, hidden_size_convL, hidden_size_end):
            super(Net, self).__init__()

            self.f1 = nn.Linear(input_size + hidden_size_convM + hidden_size_convL , hidden_size_end)
            self.sigmoid = nn.Sigmoid()
            self.f2 = nn.Linear(hidden_size_end, 2)
            self.softmax = nn.Softmax()

            self.convM = nn.Conv1d(max_event_length_week, hidden_size_convM, window_size_convM, padding = 1 )
            self.poolM = nn.MaxPool1d(input_size)

            self.convL = nn.Conv1d(max_event_length_month, hidden_size_convL, window_size_convL, padding = 1 )
            self.poolL = nn.MaxPool1d(input_size)

        def forward(self, giant_block):

            S = giant_block[:, 0,]

            M = giant_block[:,1: max_event_length_week+1,].contiguous()

            L = giant_block[:,max_event_length_week+1:,].contiguous()

            #--------------------LARGE

            out_L = self.convL(L)
            out_L = self.poolL(out_L)
            out_L = out_L.view(-1, hidden_size_convL)
            #-------------------LARGE

            #------------------- MIDDLE
            out_M = self.convM(M)
            out_M = self.poolM(out_M)
            out_M = out_M.view(-1, hidden_size_convM)
            #-------------------MIDDLE


            #x = concatenation S, M, L
            x = torch.cat((out_L, out_M,S, ), 1) 

            out = self.f1(x)
            out = self.sigmoid(out)
            out = self.f2(out)
            out = self.sigmoid(out) 
            out = self.softmax(out)
            return out 

    net = Net(input_size, window_size_convM, hidden_size_convM, window_size_convL, hidden_size_convL, hidden_size_end )

    criterion = nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

    for epoch in range(num_epochs):
        for i, (inp, outp) in enumerate(train_loader_total):

            inp = Variable(inp, requires_grad=True)
            outp = Variable(outp)

            optimizer.zero_grad()
            outputs = net(inp)
            loss = criterion(outputs, outp.squeeze()  )
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0:
            elapsed_time = time.time() - start_time
            print ("epoch:{0} elapsed_time:{1}".format(epoch, elapsed_time) + "[sec]")


    correct = 0
    total = 0
    for inp, lab in train_loader_total:
        inp = Variable(inp)

        outputs = net(inp)
        _, predicted = torch.max(outputs.data, 1)
        total += lab.size(0)
        correct += (predicted == lab).sum()


    train_accuracy = float(correct)/total 


    #--------------------------------------------------
    elapsed_time = time.time() - start_time
    print ("train_accuracy, elapsed_time:{0}".format(elapsed_time) + "[sec]")
    #--------------------------------------------------

    correct = 0
    total = 0
    for inp, lab in test_loader_total:
        inp = Variable(inp, requires_grad=True )

        outputs = net(inp)
        _, predicted = torch.max(outputs.data, 1)
        total += lab.size(0)
        correct += (predicted == lab).sum()


    test_accuracy = float(correct)/total 

    return (str(train_accuracy)[0:5], str(test_accuracy)[0:5] )

In [26]:
%%time
total_embedding_to_class('AAPL', 0.7, True)

Generated event embedded data.
elapsed_time:1.4955286979675293[sec]
epoch:0 elapsed_time:2.8046369552612305[sec]
epoch:1 elapsed_time:3.2765936851501465[sec]
epoch:2 elapsed_time:3.7584502696990967[sec]
epoch:3 elapsed_time:4.2286601066589355[sec]
epoch:4 elapsed_time:4.697137832641602[sec]
epoch:5 elapsed_time:5.166106939315796[sec]
epoch:6 elapsed_time:5.638235807418823[sec]
epoch:7 elapsed_time:6.109000205993652[sec]
epoch:8 elapsed_time:6.5731120109558105[sec]
epoch:9 elapsed_time:7.048708915710449[sec]
epoch:10 elapsed_time:7.518413782119751[sec]
epoch:11 elapsed_time:7.989004850387573[sec]
epoch:12 elapsed_time:8.460789203643799[sec]
epoch:13 elapsed_time:8.932704448699951[sec]
epoch:14 elapsed_time:9.402368307113647[sec]
epoch:15 elapsed_time:9.880853176116943[sec]
epoch:16 elapsed_time:10.348285913467407[sec]
epoch:17 elapsed_time:10.822627305984497[sec]
epoch:18 elapsed_time:11.304414510726929[sec]
epoch:19 elapsed_time:11.77756953239441[sec]
epoch:20 elapsed_time:12.248821735

epoch:179 elapsed_time:87.70165061950684[sec]
epoch:180 elapsed_time:88.16995215415955[sec]
epoch:181 elapsed_time:88.64316010475159[sec]
epoch:182 elapsed_time:89.11510133743286[sec]
epoch:183 elapsed_time:89.59092831611633[sec]
epoch:184 elapsed_time:90.06519031524658[sec]
epoch:185 elapsed_time:90.53687810897827[sec]
epoch:186 elapsed_time:91.01171731948853[sec]
epoch:187 elapsed_time:91.48676681518555[sec]
epoch:188 elapsed_time:91.96207737922668[sec]
epoch:189 elapsed_time:92.43215084075928[sec]
epoch:190 elapsed_time:92.91497802734375[sec]
epoch:191 elapsed_time:93.39013457298279[sec]
epoch:192 elapsed_time:93.86897015571594[sec]
epoch:193 elapsed_time:94.33816599845886[sec]
epoch:194 elapsed_time:94.80901503562927[sec]
epoch:195 elapsed_time:95.28243207931519[sec]
epoch:196 elapsed_time:95.75266766548157[sec]
epoch:197 elapsed_time:96.2262794971466[sec]
epoch:198 elapsed_time:96.70831990242004[sec]
epoch:199 elapsed_time:97.18016767501831[sec]
epoch:200 elapsed_time:97.660606384

epoch:355 elapsed_time:171.64308643341064[sec]
epoch:356 elapsed_time:172.0985813140869[sec]
epoch:357 elapsed_time:172.55689072608948[sec]
epoch:358 elapsed_time:173.0172119140625[sec]
epoch:359 elapsed_time:173.4896388053894[sec]
epoch:360 elapsed_time:173.95369291305542[sec]
epoch:361 elapsed_time:174.41313457489014[sec]
epoch:362 elapsed_time:174.88903188705444[sec]
epoch:363 elapsed_time:175.35464334487915[sec]
epoch:364 elapsed_time:175.81467509269714[sec]
epoch:365 elapsed_time:176.27628111839294[sec]
epoch:366 elapsed_time:176.73862648010254[sec]
epoch:367 elapsed_time:177.20370817184448[sec]
epoch:368 elapsed_time:177.67571640014648[sec]
epoch:369 elapsed_time:178.1352207660675[sec]
epoch:370 elapsed_time:178.59765887260437[sec]
epoch:371 elapsed_time:179.05725502967834[sec]
epoch:372 elapsed_time:179.5256862640381[sec]
epoch:373 elapsed_time:179.98578095436096[sec]
epoch:374 elapsed_time:180.44511151313782[sec]
epoch:375 elapsed_time:180.90668749809265[sec]
epoch:376 elapsed_

('0.999', '0.512')

In [23]:
population = ['GOOGL', 'INTC', 'AAPL', 'CSCO', 'QCOM', 'NVDA', 'AMZN', 'MSFT']
stk_lst = []

training_ratio_population = [0.1*float(i) for i in range(5,10)]
training_ratio_names = [str(0.1*float(i)) + " Training Ratio" for i in range(5,10) ]
for stock in population:
    print (stock)
    stock_length = {}
    for j in range(len(training_ratio_population)):
        stock_length[training_ratio_names[j]] = total_embedding_to_class(stock, training_ratio_population[j], True)
    stock_length["Stock Name"] = stock
    stk_lst.append(stock_length)

GOOGL
Generated event embedded data.
elapsed_time:0.3421616554260254[sec]
epoch:0 elapsed_time:0.45432496070861816[sec]
epoch:10 elapsed_time:0.8421170711517334[sec]
epoch:20 elapsed_time:1.2383525371551514[sec]
epoch:30 elapsed_time:1.6205284595489502[sec]
epoch:40 elapsed_time:2.012589931488037[sec]
epoch:50 elapsed_time:2.4055910110473633[sec]
epoch:60 elapsed_time:2.7875561714172363[sec]
epoch:70 elapsed_time:3.1777124404907227[sec]
epoch:80 elapsed_time:3.5588412284851074[sec]
epoch:90 elapsed_time:3.937565326690674[sec]
epoch:100 elapsed_time:4.323910236358643[sec]
epoch:110 elapsed_time:4.7811126708984375[sec]
epoch:120 elapsed_time:5.234429836273193[sec]
epoch:130 elapsed_time:5.654356479644775[sec]
epoch:140 elapsed_time:6.0809643268585205[sec]
epoch:150 elapsed_time:6.514720439910889[sec]
epoch:160 elapsed_time:6.9424004554748535[sec]
epoch:170 elapsed_time:7.383117914199829[sec]
epoch:180 elapsed_time:7.809109449386597[sec]
epoch:190 elapsed_time:8.25412917137146[sec]
epoch:

epoch:180 elapsed_time:13.767430067062378[sec]
epoch:190 elapsed_time:14.520787477493286[sec]
epoch:200 elapsed_time:15.238084316253662[sec]
epoch:210 elapsed_time:15.93159556388855[sec]
epoch:220 elapsed_time:16.63261318206787[sec]
epoch:230 elapsed_time:17.34425401687622[sec]
epoch:240 elapsed_time:18.093716621398926[sec]
epoch:250 elapsed_time:18.80850863456726[sec]
epoch:260 elapsed_time:19.504887342453003[sec]
epoch:270 elapsed_time:20.246013879776[sec]
epoch:280 elapsed_time:21.00184440612793[sec]
epoch:290 elapsed_time:21.723523139953613[sec]
epoch:300 elapsed_time:22.433887004852295[sec]
epoch:310 elapsed_time:23.135234832763672[sec]
epoch:320 elapsed_time:23.900086164474487[sec]
epoch:330 elapsed_time:24.666590213775635[sec]
epoch:340 elapsed_time:25.4434814453125[sec]
epoch:350 elapsed_time:26.204473972320557[sec]
epoch:360 elapsed_time:26.97394061088562[sec]
epoch:370 elapsed_time:27.7284996509552[sec]
epoch:380 elapsed_time:28.499704599380493[sec]
epoch:390 elapsed_time:29.

epoch:380 elapsed_time:12.61482286453247[sec]
epoch:390 elapsed_time:12.941048383712769[sec]
epoch:400 elapsed_time:13.26740050315857[sec]
epoch:410 elapsed_time:13.594028234481812[sec]
epoch:420 elapsed_time:13.918739557266235[sec]
epoch:430 elapsed_time:14.242030143737793[sec]
epoch:440 elapsed_time:14.569540739059448[sec]
epoch:450 elapsed_time:14.888657331466675[sec]
epoch:460 elapsed_time:15.21260929107666[sec]
epoch:470 elapsed_time:15.53762149810791[sec]
epoch:480 elapsed_time:15.85447382926941[sec]
epoch:490 elapsed_time:16.17488431930542[sec]
train_accuracy, elapsed_time:16.480971813201904[sec]
Generated event embedded data.
elapsed_time:0.2572638988494873[sec]
epoch:0 elapsed_time:0.3488173484802246[sec]
epoch:10 elapsed_time:0.7226152420043945[sec]
epoch:20 elapsed_time:1.1062169075012207[sec]
epoch:30 elapsed_time:1.4885103702545166[sec]
epoch:40 elapsed_time:1.8747446537017822[sec]
epoch:50 elapsed_time:2.2551753520965576[sec]
epoch:60 elapsed_time:2.6355860233306885[sec]


epoch:50 elapsed_time:16.583088159561157[sec]
epoch:60 elapsed_time:19.34812641143799[sec]
epoch:70 elapsed_time:21.878355503082275[sec]
epoch:80 elapsed_time:24.374887943267822[sec]
epoch:90 elapsed_time:26.903358936309814[sec]
epoch:100 elapsed_time:29.436365127563477[sec]
epoch:110 elapsed_time:31.969067811965942[sec]
epoch:120 elapsed_time:34.60242247581482[sec]
epoch:130 elapsed_time:37.51322674751282[sec]
epoch:140 elapsed_time:40.26061177253723[sec]
epoch:150 elapsed_time:43.35937213897705[sec]
epoch:160 elapsed_time:46.08033800125122[sec]
epoch:170 elapsed_time:48.76614427566528[sec]
epoch:180 elapsed_time:51.43756985664368[sec]
epoch:190 elapsed_time:54.123122215270996[sec]
epoch:200 elapsed_time:56.797537088394165[sec]
epoch:210 elapsed_time:59.69931125640869[sec]
epoch:220 elapsed_time:62.801088094711304[sec]
epoch:230 elapsed_time:65.84194874763489[sec]
epoch:240 elapsed_time:68.45368075370789[sec]
epoch:250 elapsed_time:71.0264539718628[sec]
epoch:260 elapsed_time:73.60538

epoch:250 elapsed_time:110.31431317329407[sec]
epoch:260 elapsed_time:114.27759456634521[sec]
epoch:270 elapsed_time:118.20721650123596[sec]
epoch:280 elapsed_time:122.14815545082092[sec]
epoch:290 elapsed_time:126.05544543266296[sec]
epoch:300 elapsed_time:129.98542475700378[sec]
epoch:310 elapsed_time:133.90952348709106[sec]
epoch:320 elapsed_time:137.8421790599823[sec]
epoch:330 elapsed_time:141.74626088142395[sec]
epoch:340 elapsed_time:145.65650868415833[sec]
epoch:350 elapsed_time:149.5669662952423[sec]
epoch:360 elapsed_time:153.48377108573914[sec]
epoch:370 elapsed_time:157.4018795490265[sec]
epoch:380 elapsed_time:161.3437020778656[sec]
epoch:390 elapsed_time:165.2656111717224[sec]
epoch:400 elapsed_time:169.15625762939453[sec]
epoch:410 elapsed_time:173.04015398025513[sec]
epoch:420 elapsed_time:176.99260759353638[sec]
epoch:430 elapsed_time:180.93569135665894[sec]
epoch:440 elapsed_time:184.89274311065674[sec]
epoch:450 elapsed_time:188.8353352546692[sec]
epoch:460 elapsed_t

...

epoch:330 elapsed_time:11.37961220741272[sec]
epoch:340 elapsed_time:11.72249174118042[sec]
epoch:350 elapsed_time:12.045135974884033[sec]
epoch:360 elapsed_time:12.376201868057251[sec]
epoch:370 elapsed_time:12.711185216903687[sec]
epoch:380 elapsed_time:13.056120157241821[sec]
epoch:390 elapsed_time:13.375701427459717[sec]
epoch:400 elapsed_time:13.699917316436768[sec]
epoch:410 elapsed_time:14.047612428665161[sec]
epoch:420 elapsed_time:14.396621227264404[sec]
epoch:430 elapsed_time:14.741997241973877[sec]
epoch:440 elapsed_time:15.076433658599854[sec]
epoch:450 elapsed_time:15.411340236663818[sec]
epoch:460 elapsed_time:15.736776351928711[sec]
epoch:470 elapsed_time:16.047502040863037[sec]
epoch:480 elapsed_time:16.35890531539917[sec]
epoch:490 elapsed_time:16.688130378723145[sec]
train_accuracy, elapsed_time:17.025349617004395[sec]
Generated event embedded data.
elapsed_time:0.2352457046508789[sec]
epoch:0 elapsed_time:0.30625367164611816[sec]
epoch:10 elapsed_time:0.6825044155120

epoch:50 elapsed_time:0.19930434226989746[sec]
epoch:60 elapsed_time:0.2315967082977295[sec]
epoch:70 elapsed_time:0.2645285129547119[sec]
epoch:80 elapsed_time:0.2973511219024658[sec]
epoch:90 elapsed_time:0.3308701515197754[sec]
epoch:100 elapsed_time:0.3674185276031494[sec]
epoch:110 elapsed_time:0.3986692428588867[sec]
epoch:120 elapsed_time:0.4329335689544678[sec]
epoch:130 elapsed_time:0.4651808738708496[sec]
epoch:140 elapsed_time:0.4955122470855713[sec]
epoch:150 elapsed_time:0.5267126560211182[sec]
epoch:160 elapsed_time:0.5594005584716797[sec]
epoch:170 elapsed_time:0.5912964344024658[sec]
epoch:180 elapsed_time:0.6224746704101562[sec]
epoch:190 elapsed_time:0.6556499004364014[sec]
epoch:200 elapsed_time:0.6880488395690918[sec]
epoch:210 elapsed_time:0.7201108932495117[sec]
epoch:220 elapsed_time:0.7524333000183105[sec]
epoch:230 elapsed_time:0.7842135429382324[sec]
epoch:240 elapsed_time:0.8165848255157471[sec]
epoch:250 elapsed_time:0.848257303237915[sec]
epoch:260 elapsed_

epoch:230 elapsed_time:23.696561336517334[sec]
epoch:240 elapsed_time:24.683419227600098[sec]
epoch:250 elapsed_time:25.661253929138184[sec]
epoch:260 elapsed_time:26.64557433128357[sec]
epoch:270 elapsed_time:27.816152572631836[sec]
epoch:280 elapsed_time:29.054008722305298[sec]
epoch:290 elapsed_time:30.29650568962097[sec]
epoch:300 elapsed_time:31.527390241622925[sec]
epoch:310 elapsed_time:32.615172386169434[sec]
epoch:320 elapsed_time:33.67288064956665[sec]
epoch:330 elapsed_time:34.87620186805725[sec]
epoch:340 elapsed_time:35.96711802482605[sec]
epoch:350 elapsed_time:37.16710543632507[sec]
epoch:360 elapsed_time:38.39910554885864[sec]
epoch:370 elapsed_time:39.489205837249756[sec]
epoch:380 elapsed_time:40.54520845413208[sec]
epoch:390 elapsed_time:41.60715460777283[sec]
epoch:400 elapsed_time:42.7934935092926[sec]
epoch:410 elapsed_time:43.910707235336304[sec]
epoch:420 elapsed_time:44.900444746017456[sec]
epoch:430 elapsed_time:45.89974880218506[sec]
epoch:440 elapsed_time:46

epoch:430 elapsed_time:79.21050834655762[sec]
epoch:440 elapsed_time:81.13920950889587[sec]
epoch:450 elapsed_time:83.10740041732788[sec]
epoch:460 elapsed_time:84.8323073387146[sec]
epoch:470 elapsed_time:86.54154324531555[sec]
epoch:480 elapsed_time:88.2339780330658[sec]
epoch:490 elapsed_time:90.17114400863647[sec]
train_accuracy, elapsed_time:91.98514986038208[sec]
Generated event embedded data.
elapsed_time:0.7031888961791992[sec]
epoch:0 elapsed_time:1.1219253540039062[sec]
epoch:10 elapsed_time:2.9365155696868896[sec]
epoch:20 elapsed_time:5.009688854217529[sec]
epoch:30 elapsed_time:6.9444661140441895[sec]
epoch:40 elapsed_time:9.114806652069092[sec]
epoch:50 elapsed_time:11.329510688781738[sec]
epoch:60 elapsed_time:13.250307559967041[sec]
epoch:70 elapsed_time:15.131446361541748[sec]
epoch:80 elapsed_time:17.280905723571777[sec]
epoch:90 elapsed_time:19.4994797706604[sec]
epoch:100 elapsed_time:21.701537132263184[sec]
epoch:110 elapsed_time:23.918835163116455[sec]
epoch:120 e

epoch:110 elapsed_time:12.12282133102417[sec]
epoch:120 elapsed_time:13.167349815368652[sec]
epoch:130 elapsed_time:14.2009117603302[sec]
epoch:140 elapsed_time:15.23309874534607[sec]
epoch:150 elapsed_time:16.26355266571045[sec]
epoch:160 elapsed_time:17.397813320159912[sec]
epoch:170 elapsed_time:18.65604257583618[sec]
epoch:180 elapsed_time:19.935578107833862[sec]
epoch:190 elapsed_time:21.2126202583313[sec]
epoch:200 elapsed_time:22.43314290046692[sec]
epoch:210 elapsed_time:23.679032802581787[sec]
epoch:220 elapsed_time:24.93921661376953[sec]
epoch:230 elapsed_time:26.209492921829224[sec]
epoch:240 elapsed_time:27.491031169891357[sec]
epoch:250 elapsed_time:28.761279344558716[sec]
epoch:260 elapsed_time:29.895127534866333[sec]
epoch:270 elapsed_time:31.127336978912354[sec]
epoch:280 elapsed_time:32.40730023384094[sec]
epoch:290 elapsed_time:33.70669102668762[sec]
epoch:300 elapsed_time:34.99075150489807[sec]
epoch:310 elapsed_time:36.26848483085632[sec]
epoch:320 elapsed_time:37.4

In [24]:
df = pd.DataFrame(stk_lst)
df[["Stock Name"] + training_ratio_names  ]

,Stock Name,0.5 Training Ratio,0.6000000000000001 Training Ratio,0.7000000000000001 Training Ratio,0.8 Training Ratio,0.9 Training Ratio
0,GOOGL,"(1.0, 0.510)","(1.0, 0.516)","(0.996, 0.448)","(0.996, 0.428)","(0.991, 0.5)"
1,INTC,"(0.993, 0.525)","(0.989, 0.578)","(0.977, 0.479)","(0.968, 0.468)","(0.982, 0.375)"
2,AAPL,"(1.0, 0.471)","(1.0, 0.507)","(1.0, 0.467)","(1.0, 0.478)","(1.0, 0.525)"
3,CSCO,"(0.989, 0.549)","(0.981, 0.534)","(0.960, 0.545)","(0.959, 0.555)","(0.957, 0.611)"
4,QCOM,"(1.0, 0.508)","(0.985, 0.526)","(0.981, 0.565)","(0.989, 0.565)","(0.985, 0.565)"
5,NVDA,"(1.0, 0.571)","(1.0, 0.545)","(1.0, 0.75)","(1.0, 0.8)","(1.0, 1.0)"
6,AMZN,"(0.994, 0.545)","(0.997, 0.498)","(0.986, 0.530)","(0.983, 0.446)","(0.991, 0.546)"
7,MSFT,"(0.993, 0.5)","(1.0, 0.452)","(0.990, 0.494)","(0.995, 0.512)","(0.998, 0.466)"


In [29]:
def debug_total_embedding_to_class(stock, training_ratio, shuff_bool): 


    start_time = time.time()

    #----------------------------LAGS
    day_lag = 1 
    week_lag = 7
    month_lag = 30
    #--------------------------- NN parameters
    input_size = 100
    window_size_convM =3
    hidden_size_convM = 20

    window_size_convL =3 
    hidden_size_convL = 40


    hidden_size_end = 200

    learning_rate = 0.001
    #num_epochs = 500
    num_epochs = 10
    #batch_size = 50
    batch_size = 10

    #training_ratio = 0.8
    #stock = 'AAPL'

    #----------------------------------- PARAMTETRS


    stock_price_csv = pd.read_csv(repo_dir + "/price_data/"+ stock+"_2006-01-01_to_2017-11-01.csv")

    temp_x = []
    temp_y = []
    for i in range(len(stock_to_events[stock]) ):
        event = stock_to_events[stock][i]


        date_numeric = event[0]
        day = conv_num_to_string( date_numeric    )


        if day in stock_price_csv["Date"].values:

            temp = {}
            temp["day"] = embedding_lst[event[1]]



            row_index = stock_price_csv.index[stock_price_csv["Date"] == day].tolist()[0]
            next_price_data = stock_price_csv.iloc[row_index - day_lag  ]

            next_price = next_price_data["Close"] - next_price_data["Open"]

            if next_price >= 0.0:
                pos_neg_class = 1
            else:
                pos_neg_class = 0
            temp_y.append(pos_neg_class)



            temp["week"] = []
            temp_date_before = 1
            while ( i - temp_date_before >= 0 and numeric_day_distance(stock_to_events[stock][i-temp_date_before][0], date_numeric ) <= week_lag ) :
                temp['week'].append(embedding_lst[stock_to_events[stock][i-temp_date_before][1]]    )
                temp_date_before +=1 

            temp["month"] = []
            temp_date_before = 1
            while ( i - temp_date_before >= 0 and numeric_day_distance(stock_to_events[stock][i-temp_date_before][0], date_numeric ) <= month_lag ) :
                temp['month'].append(embedding_lst[stock_to_events[stock][i-temp_date_before][1]]    )
                temp_date_before +=1 
            temp_x.append(temp)
    #--------------------------------------

    print ("Generated event embedded data.")
    elapsed_time = time.time() - start_time
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

    print ("X")
    print (temp_x[:5])
    print ("Y")
    print (temp_y[:5])

    #--------------------------
    sample_size = len(temp_x)
    cut = int(training_ratio*float(sample_size) ) 
    train_x = temp_x[0:cut ]
    test_x = temp_x[cut+1:]

    train_y = temp_y[0:cut]
    test_y = temp_y[cut+1:]


    max_event_length_week = max([len(day_embedding["week"]) for day_embedding in temp_x ])
    max_event_length_month = max([len(day_embedding["month"]) for day_embedding in temp_x ])

    print ("max_event_length_week: {0}".format(max_event_length_week))
    print ("max_event_length_month: {0}".format(max_event_length_month))

    train_x_concatenate = []



    for day_embedding in train_x: 
        block = [day_embedding["day"]]
        week_padding_number = max_event_length_week - len(day_embedding["week"])
        block = block + day_embedding["week"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(week_padding_number)]

        month_padding_number = max_event_length_month - len(day_embedding["month"])
        block = block + day_embedding["month"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(month_padding_number)]


        train_x_concatenate.append(block)

    train_x_concatenate = torch.FloatTensor(train_x_concatenate)

    train_y = torch.LongTensor(train_y)
    train_x_concatenate_temp = torch.utils.data.TensorDataset(train_x_concatenate, train_y)
    train_loader_total = torch.utils.data.DataLoader(dataset=train_x_concatenate_temp, 
                                           batch_size = batch_size, 
                                           shuffle=shuff_bool)



    #-----------------------------------

    test_x_concatenate = []


    for day_embedding in test_x: 
        block = [day_embedding["day"]]
        week_padding_number = max_event_length_week - len(day_embedding["week"])
        block = block + day_embedding["week"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(week_padding_number)]

        month_padding_number = max_event_length_month - len(day_embedding["month"])
        block = block + day_embedding["month"]
        block = block + [[0.0 for i in range(input_size) ] for j in range(month_padding_number)]


        test_x_concatenate.append(block)


    test_x_concatenate = torch.FloatTensor(test_x_concatenate)

    test_y = torch.LongTensor(test_y)
    test_x_concatenate_temp = torch.utils.data.TensorDataset(test_x_concatenate, test_y)
    test_loader_total = torch.utils.data.DataLoader(dataset=test_x_concatenate_temp, 
                                           batch_size = batch_size, 
                                           shuffle=False)



    #------------------------------------------
    class Net(nn.Module):

        def __init__(self, input_size, window_size_convM, hidden_size_convM, window_size_convL, hidden_size_convL, hidden_size_end):
            super(Net, self).__init__()

            self.f1 = nn.Linear(input_size + hidden_size_convM + hidden_size_convL , hidden_size_end)
            self.sigmoid = nn.Sigmoid()
            self.f2 = nn.Linear(hidden_size_end, 2)
            self.softmax = nn.Softmax()

            self.convM = nn.Conv1d(max_event_length_week, hidden_size_convM, window_size_convM, padding = 1 )
            self.poolM = nn.MaxPool1d(input_size)

            self.convL = nn.Conv1d(max_event_length_month, hidden_size_convL, window_size_convL, padding = 1 )
            self.poolL = nn.MaxPool1d(input_size)

        def forward(self, giant_block):

            S = giant_block[:, 0,]

            M = giant_block[:,1: max_event_length_week+1,].contiguous()

            L = giant_block[:,max_event_length_week+1:,].contiguous()

            #--------------------LARGE

            out_L = self.convL(L)
            out_L = self.poolL(out_L)
            out_L = out_L.view(-1, hidden_size_convL)
            #-------------------LARGE

            #------------------- MIDDLE
            out_M = self.convM(M)
            out_M = self.poolM(out_M)
            out_M = out_M.view(-1, hidden_size_convM)
            #-------------------MIDDLE


            #x = concatenation S, M, L
            x = torch.cat((out_L, out_M,S, ), 1) 

            out = self.f1(x)
            out = self.sigmoid(out)
            out = self.f2(out)
            out = self.sigmoid(out) 
            out = self.softmax(out)
            return out 

    net = Net(input_size, window_size_convM, hidden_size_convM, window_size_convL, hidden_size_convL, hidden_size_end )

    criterion = nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

    for epoch in range(num_epochs):
        for i, (inp, outp) in enumerate(train_loader_total):

            inp = Variable(inp, requires_grad=True)
            outp = Variable(outp)

            optimizer.zero_grad()
            outputs = net(inp)
            loss = criterion(outputs, outp.squeeze()  )
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0:
            elapsed_time = time.time() - start_time
            print ("epoch:{0} elapsed_time:{1}".format(epoch, elapsed_time) + "[sec]")


    correct = 0
    total = 0
    for inp, lab in train_loader_total:
        inp = Variable(inp)

        outputs = net(inp)
        _, predicted = torch.max(outputs.data, 1)
        total += lab.size(0)
        correct += (predicted == lab).sum()


    train_accuracy = float(correct)/total 


    #--------------------------------------------------
    elapsed_time = time.time() - start_time
    print ("train_accuracy, elapsed_time:{0}".format(elapsed_time) + "[sec]")
    #--------------------------------------------------

    correct = 0
    total = 0
    for inp, lab in test_loader_total:
        inp = Variable(inp, requires_grad=True )

        outputs = net(inp)
        _, predicted = torch.max(outputs.data, 1)
        total += lab.size(0)
        correct += (predicted == lab).sum()


    test_accuracy = float(correct)/total 

    return (str(train_accuracy)[0:5], str(test_accuracy)[0:5] )

In [30]:
debug_total_embedding_to_class('AAPL', 0.7, True)

Generated event embedded data.
elapsed_time:1.4924497604370117[sec]
X
[{'day': [-1.1198910474777222, 0.6199478507041931, 1.503580927848816, -0.5412606596946716, 0.8735957145690918, 0.8843127489089966, -0.7662367224693298, 1.2859549522399902, 1.0485315322875977, 1.5888556241989136, 0.832848072052002, -0.6466353535652161, -1.314500331878662, 0.7553664445877075, -0.8322042226791382, 2.0567455291748047, 0.8449605703353882, 1.7911388874053955, -0.6625466346740723, 1.0547263622283936, -1.7043612003326416, -1.1597107648849487, -1.5861331224441528, 1.3258944749832153, -0.99997478723526, -1.3683515787124634, 0.48563307523727417, -1.1355892419815063, 1.862587332725525, -0.7663973569869995, -1.0352706909179688, 0.4348338544368744, 1.3240712881088257, -0.9140188694000244, -0.26349419355392456, -0.49873673915863037, -0.7301521301269531, -0.9356916546821594, -1.6626167297363281, 0.8540869355201721, -0.9384216666221619, -0.7821177840232849, -0.7363995313644409, 0.6167747974395752, 0.45744800567626953

epoch:0 elapsed_time:2.893005609512329[sec]
train_accuracy, elapsed_time:7.904895067214966[sec]


('0.981', '0.489')